In [1]:
import ifcopenshell

model = ifcopenshell.open("../data/sample.ifc")
print("Modelo cargado correctamente")


Modelo cargado correctamente


In [2]:
# Obtener todos los espacios del modelo IFC
spaces = model.by_type("IfcSpace")

print(f"Número total de espacios encontrados: {len(spaces)}")

for i, space in enumerate(spaces[:10], start=1):
    name = space.LongName or space.Name
    print(f"{i}. {name}")


Número total de espacios encontrados: 21
1. Foyer
2. Hallway
3. Bathroom 1
4. Foyer
5. Hallway
6. Utility
7. Utility
8. Stair
9. Room
10. Roof


In [3]:
# Crear un resumen textual del proyecto (input para GenAI)
space_names = []

for space in spaces:
    name = space.LongName or space.Name
    if name:
        space_names.append(name)

summary_text = (
    f"Este proyecto inmobiliario contiene {len(space_names)} espacios. "
    f"Los principales ambientes identificados son: "
    + ", ".join(space_names[:10])
)

print(summary_text)


Este proyecto inmobiliario contiene 21 espacios. Los principales ambientes identificados son: Foyer, Hallway, Bathroom 1, Foyer, Hallway, Utility, Utility, Stair, Room, Roof


In [4]:
walls = model.by_type("IfcWall")
doors = model.by_type("IfcDoor")
windows = model.by_type("IfcWindow")

print(f"Muros: {len(walls)}")
print(f"Puertas: {len(doors)}")
print(f"Ventanas: {len(windows)}")


Muros: 57
Puertas: 14
Ventanas: 24


In [5]:
project_summary = f"""
Resumen del proyecto inmobiliario:

- Número total de espacios: {len(spaces)}
- Número de muros: {len(walls)}
- Número de puertas: {len(doors)}
- Número de ventanas: {len(windows)}

Este resumen fue generado automáticamente a partir de un archivo IFC,
con el objetivo de permitir consultas en lenguaje natural usando GenAI.
"""

print(project_summary)



Resumen del proyecto inmobiliario:

- Número total de espacios: 21
- Número de muros: 57
- Número de puertas: 14
- Número de ventanas: 24

Este resumen fue generado automáticamente a partir de un archivo IFC,
con el objetivo de permitir consultas en lenguaje natural usando GenAI.



Eres un asistente experto en proyectos inmobiliarios y construcción.

A continuación se te proporciona información técnica extraída automáticamente
desde un archivo IFC de un proyecto inmobiliario.

Tu tarea es:
- Interpretar esta información técnica
- Responder preguntas en lenguaje natural de forma clara y comprensible
- No inventar datos que no estén presentes en la información proporcionada

Información del proyecto:
---
Resumen del proyecto inmobiliario:

- Número total de espacios: 21
- Número de muros: 57
- Número de puertas: 14
- Número de ventanas: 24

Este resumen fue generado automáticamente a partir de un archivo IFC,
con el objetivo de permitir consultas en lenguaje natural usando GenAI.
---

Instrucciones adicionales:
- Si una pregunta no puede responderse con la información disponible, indícalo claramente.
- Usa un lenguaje claro, orientado a usuarios no técnicos.
- Prioriza explicaciones concisas pero completas.

Responde la siguiente pregunta:

¿Puedes darme un resumen general del proyecto inmobiliario?
